In [1]:
import cv2
from keras.models import load_model
from mtcnn.mtcnn import MTCNN
import numpy as np

Using TensorFlow backend.
/home/ketan/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ketan/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ketan/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ketan/.local/lib/python3.6/site-packages/tensorflow/python/fra

In [11]:
class detection:
    def __init__(self):
        # model to fing face embedding
        self.embeddings = load_model('facenet_keras.h5')
        # model to detect faces
        self.face_detector = MTCNN()
        self.tolerance = 0.6
        self.wanted = []
        
    def update_wanted(self, image):
        faces, coordinates = self.find_faces(image)
        print(faces)
        encoding = self.find_embeddings(faces)
        print('encoding: ', encoding)
        self.wanted.append(encoding[0])
        print("encoding appended")
        
    def find_faces(self, image):
        faces = self.face_detector.detect_faces(image)
        if len(faces) > 0:
            print('face detected')
        face_images = []
        coordinates = []
        for face in faces:
            [x, y, w, h] = face['box']
            coordinates.append(face['box'])
            face_images.append(cv2.resize(image[y:y+h, x:x+w], (160, 160)))
        return face_images, coordinates

    def find_embeddings(self, list_faces):
        embedding_list = []
        if len(list_faces) == 0:
            print("no face detected!!!")
            return embedding_list
        else:
            print('inside embeddings')
        for face in list_faces:
            print(face.shape)
            mean, std = face.mean(), face.std()
            face = (face - mean) / std
            print(face.shape)
            samples = np.expand_dims(face, axis=0)
            print(samples.shape)
            embedding_list.append(self.embeddings.predict(samples))
        return embedding_list
    
    def compare_faces(self, face_embeddings): # compare the encodings in wanted list with the face and 
        print("compare function called.")
        if len(self.wanted) == 0:
            print("wanted empty!")
            return
        if len(face_embeddings) == 0:
            print("no face encoding present!!!")
        for index, wanted_encoding in enumerate(self.wanted):
            distances = np.linalg.norm(face_embeddings - wanted_encoding, axis=1)
            print("distance: ", distances)
            for distance in distances:
                print(distance)
                if distance <= self.tolerance:
                    
                    print("match found for wanted number"+str(index))

In [12]:
d = detection()
print('detector created.')
# Adding wanted
cam = cv2.VideoCapture(0)

while(1):
    ok, frame = cam.read()
    if ok:
        cv2.imshow("Live Image.", frame)
        k = cv2.waitKey(0)
        if k & 0xFF == ord('q'):
            print('Updating wanted.')
            d.update_wanted(frame)
            print("Wanted updated.")
            cv2.destroyAllWindows()
            cam.release()
            break
        

detector created.


In [14]:
cam = cv2.VideoCapture(0)

while 1:
    ok, frame = cam.read()
    if ok:
        faces, coordinated = d.find_faces(frame)
        encodings = d.find_embeddings(faces)
        d.compare_faces(encodings)
        # print(encodings)
        cv2.imshow("live image", frame)
        for index, face in enumerate(faces):
            cv2.imshow("face "+str(index), face)
        k = cv2.waitKey(2)
        if k & 0xff == ord('q'):
            cv2.destroyAllWindows()
            cam.release()
            break
    else:
        print('camera not working')

face detected
inside embeddings
(160, 160, 3)
(160, 160, 3)
(1, 160, 160, 3)
(160, 160, 3)
(160, 160, 3)
(1, 160, 160, 3)
compare function called.
distance:  [[0.14570059 0.17809272 0.05709571 0.5558807  0.18414223 0.17596757
  0.37290776 0.22933078 0.05360308 0.36506867 0.27751815 0.2044189
  0.46962    0.17420065 0.3954608  0.06570965 0.93173337 0.23098844
  0.04709263 0.39237309 0.46631813 0.0469256  0.25855038 0.18298423
  0.8350464  0.33417994 0.17228138 0.3006441  0.28638005 0.31004828
  0.29521558 0.14295042 0.6749835  0.34575975 0.0298959  0.03312743
  0.27622426 0.7100376  0.32151198 0.48661345 0.23546082 0.15775731
  0.16914731 0.12561327 0.11522883 0.09703893 0.09897184 0.21439505
  0.35306454 0.5245637  0.0467183  0.21598953 0.5609143  0.45351437
  0.57385445 0.3435911  0.04340942 0.82730806 0.4702723  0.42155468
  0.4428609  0.60381424 0.09141248 0.46465632 0.093858   0.15747082
  0.12627918 0.60678226 0.51075196 0.02874219 0.70927477 0.4473677
  0.5980085  0.19251221 0.18

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

Updating wanted.
face detected
[array([[[140, 139, 141],
        [141, 141, 141],
        [142, 142, 142],
        ...,
        [123, 106, 111],
        [122, 109, 114],
        [124, 111, 116]],

       [[136, 136, 136],
        [139, 138, 138],
        [142, 140, 141],
        ...,
        [129, 109, 114],
        [129, 113, 116],
        [132, 115, 119]],

       [[133, 132, 133],
        [138, 136, 138],
        [142, 139, 140],
        ...,
        [131, 110, 113],
        [133, 114, 117],
        [135, 117, 120]],

       ...,

       [[221, 234, 234],
        [220, 233, 233],
        [220, 233, 233],
        ...,
        [221, 228, 228],
        [221, 228, 228],
        [221, 228, 227]],

       [[220, 233, 233],
        [220, 233, 233],
        [220, 233, 233],
        ...,
        [222, 229, 229],
        [222, 228, 230],
        [221, 227, 229]],

       [[220, 233, 233],
        [220, 233, 233],
        [220, 233, 233],
        ...,
        [223, 230, 231],
        [222, 230

In [15]:
cam.release()

In [ ]:
cam = cv2.VideoCapture(0)

while 1:
    ok, frame = cam.read()
    if ok:
        faces = d.face_detector.detect_faces(frame)
        for face in faces:
            [x,y,w,h] = face['box']
            cv2.rectangle(frame, (x,y), (x+w, y+h), (255,0,0), 2)
        cv2.imshow("live feed", frame)
        k = cv2.waitKey(2)
        if k & 0xff == ord('q'):
            cv2.destroyAllWindows()
            cam.release()
            break
    else:
        print('camera not working')